Problem Statement.

Given an array of strings products and a string searchWord. We want to design a system that suggests at most three product names from products after each character of searchWord is typed. Suggested products should have common prefix with the searchWord. If there are more than three products with a common prefix return the three lexicographically minimums products.

Return list of lists of the suggested products after each character of searchWord is typed. 

 

Example 1:

Input: products = ["mobile","mouse","moneypot","monitor","mousepad"], searchWord = "mouse"
Output: [
["mobile","moneypot","monitor"],
["mobile","moneypot","monitor"],
["mouse","mousepad"],
["mouse","mousepad"],
["mouse","mousepad"]
]
Explanation: products sorted lexicographically = ["mobile","moneypot","monitor","mouse","mousepad"]
After typing m and mo all products match and we show user ["mobile","moneypot","monitor"]
After typing mou, mous and mouse the system suggests ["mouse","mousepad"]

Example 2:

Input: products = ["havana"], searchWord = "havana"
Output: [["havana"],["havana"],["havana"],["havana"],["havana"],["havana"]]

Example 3:

Input: products = ["bags","baggage","banner","box","cloths"], searchWord = "bags"
Output: [["baggage","bags","banner"],["baggage","bags","banner"],["baggage","bags"],["bags"]]

Example 4:

Input: products = ["havana"], searchWord = "tatiana"
Output: [[],[],[],[],[],[],[]]

 

Constraints:

    1 <= products.length <= 1000
    There are no repeated elements in products.
    1 <= Σ products[i].length <= 2 * 10^4
    All characters of products[i] are lower-case English letters.
    1 <= searchWord.length <= 1000
    All characters of searchWord are lower-case English letters.

# Heap - O(P * N ^ 2 + S ^ 2) runtime, O(P * N) space, where P is no. of products, N is the max length of each product and S is the length of the searchword 

In [1]:
from typing import List
from collections import defaultdict
from heapq import heappush, heappop

class Solution:
    def suggestedProducts(self, products: List[str], searchWord: str) -> List[List[str]]:
        wordDict = defaultdict(list)
        
        for product in products:
            prefix = ''
            for c in product:
                prefix += c
                heappush(wordDict[prefix], product)
        
        result = []
        prefix = ''
        for c in searchWord:
            prefix += c
            if prefix not in wordDict: result.append([])
            else:
                newList = []
                j = 0
                while wordDict[prefix] and j < 3:
                    newList.append(heappop(wordDict[prefix]))
                    j += 1
                result.append(newList)
        
        return result

# Trie with Heap - O(N + S) runtime, O(P*N + S) space

In [2]:
from typing import List
from collections import defaultdict
from heapq import heappush, heappushpop

class Solution:
    def suggestedProducts(self, products: List[str], searchWord: str) -> List[List[str]]:
        class TrieNode:
            def __init__(self):
                self.children = defaultdict(TrieNode)
                self.h = []
            
            def add_sugesstion(self, product):
                if len(self.h) < 3:
                    heappush(self.h, MaxHeapStr(product))
                else:
                    heappushpop(self.h, MaxHeapStr(product))
            
            def get_suggestion(self):
                return sorted(self.h, reverse = True)
        
        class MaxHeapStr(str):
            def __init__(self, string): self.string = string
            def __lt__(self,other): return self.string > other.string
            def __eq__(self,other): return self.string == other.string
        
        root = TrieNode()
        for p in products:
            node = root
            for char in p:
                node = node.children[char]
                node.add_sugesstion(p)
        
        result, node = [], root
        for char in searchWord:
            node = node.children[char]
            result.append(node.get_suggestion())
        return result

# Sort and Two Pointer - O(P Log P + W * P) runtime, O(P) space

In [1]:
from typing import List

class Solution:
    def suggestedProducts(self, products: List[str], searchWord: str) -> List[List[str]]:
        n = len(products)
        products.sort()  # Sort by increasing lexicographically order of products
        ans = []
        startIdx, endIdx = 0, n - 1
        for i, c in enumerate(searchWord):
            while startIdx <= endIdx and (i >= len(products[startIdx]) or products[startIdx][i] < c):
                startIdx += 1
            while startIdx <= endIdx and (i >= len(products[endIdx]) or products[endIdx][i] > c):
                endIdx -= 1

            if startIdx <= endIdx:
                ans.append(products[startIdx:min(startIdx+3, endIdx+1)])
            else:
                ans.append([])
        return ans

# Binary Search - O(P Log P) + O(S Log P) runtime, O(P + S) space

In [9]:
from typing import List
from bisect import bisect_left

class Solution:
    def suggestedProducts(self, products: List[str], searchWord: str) -> List[List[str]]:
        products.sort()
        res, prefix, i = [], '', 0
        for c in searchWord:
            prefix += c
            i = bisect_left(products, prefix, i)
            res.append([w for w in products[i:i + 3] if w.startswith(prefix)])
        return res

# Trie - O(M) to build the trie where M is total number of characters in products runtime, O(26 * N) where N is the number of nodes in the trie

In [7]:
from typing import List

class Solution:
    def suggestedProducts(self, products: List[str], searchWord: str) -> List[List[str]]:
        trie= {}

        for p in products:
            cur = trie
            for c in p:
                if c not in cur: cur[c] = {}
                cur = cur[c]
            if '#' not in cur: cur['#'] = True

        def search_word(cur, cur_list, cur_word):
            if len(cur_list) == 3: return
            if '#' in cur:
                cur_list.append(''.join(cur_word))
            for char in 'abcdefghijklmnopqrstuvwxyz':
                if char in cur:
                    search_word(cur[char], cur_list, cur_word + [char])
        
        result = []
        cur = trie
        n = len(searchWord)
        word_so_far = []
        for i, s in enumerate(searchWord):
            if s not in cur:
                for _ in range(i, n):
                    result.append([])
                break
            word_so_far.append(s) 
            cur = cur[s]
            cur_list = list()
            search_word(cur, cur_list, list(word_so_far))
            result.append(cur_list)

        return result

In [11]:
instance = Solution()
instance.suggestedProducts(["mobile","mouse","moneypot","monitor","mousepad"], "mouse")

[['mobile', 'moneypot', 'monitor'],
 ['mobile', 'moneypot', 'monitor'],
 ['mouse', 'mousepad'],
 ['mouse', 'mousepad'],
 ['mouse', 'mousepad']]